## Census Annual Business Survey ETL:

#### Company Summary 

The first step in our ETL report was to import the modules that were necessary to pull in our dataset. We used the **requests** module to help us send HTTP requests to the Census API's endpoint and retrieve the data. The **pandas** module is used to create a new data frame structure in which we can place and manipulate our data.

In [1]:
import requests 
import pandas as pd


The next step is to create a list of *useful_columns* that we want to use as our desired data frame columns. The *get* variable is composed of the dataset columns that we will be pulling from. *get_list* splits *get* on every instance of a comma. 

In [4]:
useful_columns = ['NAME', 'NAICS2017_LABEL','RCPSZFI_LABEL', 'SEX_LABEL', 'ETH_GROUP_LABEL', 'RACE_GROUP_LABEL', 'VET_GROUP_LABEL', 'EMPSZFI_LABEL', 'FIRMPDEMP', 'RCPPDEMP', 'PAYANN', 'YEAR', "YIBSZFI", "EMP"]
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,YIBSZFI,PAYANN,EMP,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,RCPSZFI_LABEL,PAYANN_S,PAYANN_S_F,RCPSZFI'
get_list = get.split(',')

We now need to drop the columns that aren't needed for our data frame. We created an empty list called *columns_to_drop*. We then created a for loop that iterates through *get_list* from above and checks to see if the column is a part of *useful_columns*. We then add the columns not a part of *useful_columns* to our empty list.

In [5]:
# Appending the less useful columns
columns_to_drop = []
for items in get_list:
    if items in useful_columns:
        pass
    else:
        columns_to_drop.append(items)
# When one adds filters to the api, it adds duplicates of the columns to the data.
# Instead of renaming those columns (which are auto-filled in Pandas as integers)
# we will append the colmun number to drop it later
columns_to_drop.append(31) 

Now, we will pull the data from the Census. We use a loop to iterate through all the years of data available. We use our API key and census data url to run a get request on the dataset. We rename our headers, and drop the unecessary columns in the *columns_to_drop* list. Our final data frame, *company_summary* is created at the end of the code. 

In [6]:
for i in range(2017, 2021):
    # We establish a base year
    if i == 2017:
        year = str(i)
        key = "be6437ff702ba92a0aebb292fab267af87d8549c"
        geography = 'us'
        url = (f"https://api.census.gov/data/{year}/abscs?get={get}" +
                f"&for={geography}:" +
                f"*&key={key}")
        r = requests.get(url)
        text = r.text

        # We create a temporary database using the text
        df = pd.read_json(text)

        # We're now appending the first row of df to an empty list, headers
        headers = []
        for i in range(0,len(get_list)):
            headers.append(df.iloc[0][i])

        # Now we will extract the remaining data from df
        company_summary = pd.DataFrame(data = df.iloc[1:])

        # We will next reset the headers of company_summary
        for i in range(0,len(get_list)):
            company_summary.rename(columns = {i: headers[i]}, inplace = True)

        # Now we will drop the less useful columns
        company_summary.drop(columns_to_drop, axis = 1, inplace = True)
    else:
        # We repeat the previous setps fro next year's data
        year = str(i)
        key = "be6437ff702ba92a0aebb292fab267af87d8549c"
        geography = 'us'
        url = (f"https://api.census.gov/data/{year}/abscs?get={get}" +
                f"&for={geography}:" +
                f"*&key={key}")
        r = requests.get(url)
        text = r.text

        df = pd.read_json(text)

        company_summary_tmp = pd.DataFrame(data = df.iloc[1:])

        for i in range(0,len(get_list)):
            company_summary_tmp.rename(columns = {i: headers[i]}, inplace = True)
        company_summary_tmp.drop(columns_to_drop, axis = 1, inplace = True)

        # Now we will append our dataframe with next year's data
        company_summary = pd.DataFrame(company_summary.append(company_summary_tmp, ignore_index=True))

C:\Users\MasonLonoff\AppData\Local\Temp\ipykernel_23580\292551875.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  company_summary = pd.DataFrame(company_summary.append(company_summary_tmp, ignore_index=True))
C:\Users\MasonLonoff\AppData\Local\Temp\ipykernel_23580\292551875.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  company_summary = pd.DataFrame(company_summary.append(company_summary_tmp, ignore_index=True))
C:\Users\MasonLonoff\AppData\Local\Temp\ipykernel_23580\292551875.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  company_summary = pd.DataFrame(company_summary.append(company_summary_tmp, ignore_index=True))


We now have our data frame with our selected columns. The last step of cleaning our data is to make sure the data have the correct data types. The code below changes certain columns into the correct data type. 

In [8]:
company_summary['YEAR'] = company_summary['YEAR'].astype(int)
company_summary['FIRMPDEMP'] = company_summary['FIRMPDEMP'].astype(int)
company_summary['RCPPDEMP'] = company_summary['RCPPDEMP'].astype(float)
company_summary['EMP'] = company_summary['EMP'].astype(int)
company_summary['PAYANN'] = company_summary['PAYANN'].astype(float)

Finally, we need to save our dataframe so it can be used for further analysis.

In [12]:
company_summary.to_csv("company_summary_data.csv", mode = 'w')

We have now created a new data frame called *company_summary* that has been cleaned and is now ready to be used for visualizations

##### Characteristics of Business Owners 

First, let's import in the necessary modules to pull in the dataset. The **requests** module is needed for the web scrapping via a Census API. The **pandas** module is used to provide the data frame structure in which data is placed in. Four helper functions are also created to extract, transform and load the Census data.

In [ ]:
import requests
import pandas as pd

In [ ]:
get ="GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F"
get_list = get.split(',')

key = "27133f99dc56a667684ae931bb15f0c829847d6a"
geography = 'us' 
columns_to_drop = ['GEO_ID', 'NAICS2017', 'OWNER_SEX', 'OWNER_ETH', 'OWNER_RACE', 'OWNER_VET', 'QDESC', 'OWNCHAR', 'OWNPDEMP_PCT', 'OWNPDEMP_PCT_F', 'OWNPDEMP_S', 'OWNPDEMP_S_F', 'OWNPDEMP_PCT_S', 'OWNPDEMP_PCT_S_F', 'us']

decoder1 = {'NAME': 'Location',
           'NAICS2017_LABEL':'Business_Sector',
           'OWNER_SEX_LABEL':'Owner_Sex',
           'OWNER_ETH_LABEL':'Owner_Ethnicity',
           'OWNER_RACE_LABEL':'Owner_Race',
           'OWNER_VET_LABEL':'Veteran_Status',
           'QDESC_LABEL':'Question',
           'OWNCHAR_LABEL':'Owner_Response',
           'YEAR':'Year',
           'OWNPDEMP':'Number_of_Responses',
            'OWNPDEMP_F':'Response_Footnotes'}

def restructure(df):
    temp_data= pd.DataFrame(data = df.iloc[1:])
    for i in range(len(temp_data.columns)):
        temp_data.rename(columns = {i:df.iloc[0][i]}, inplace = True)
    return temp_data

def extract_data(year):
    url = (f"https://api.census.gov/data/{year}/abscbo?get={get}" +
        f"&for={geography}:" +
        f"*&key={key}")
    r = requests.get(url)
    text = r.text
    data_f = pd.read_json(text)
    df = restructure(data_f)
    df.to_csv("characteristics-of-business-owners-" + str(year) + ".csv", mode = 'w')
    print(f'Data saved for {year}.')

def load_data(year_clean):
    df = pd.read_csv('characteristics-of-business-owners-' + str(year_clean) + '.csv')
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    print(f'{year_clean} data has been loaded.')
    return df

def clean_data(dataframe,year):
    dataframe.drop(columns_to_drop, axis = 1, inplace = True)
    for key, item in decoder1.items():
        dataframe.rename(columns = {key: item}, inplace = True)
    dataframe.to_csv("characteristics-of-business-owners-" + str(year) + "_clean.csv", mode = 'w')
    print(f'{year} data has been cleaned,saved and ready to be loaded for use.')
    

To pull the data from the Census for each year between 2017 to 2020, the *extract_data* function was used to place in the desired variables (which are all of them at this point) into the URL, along with the API Key. Then it uses **request** to pull in data into a json style text and changes it to a data frame. The data frame is then passed through the *restructure* function which takes into account that the first row of the data are column names, not actual data. This data is then saved onto a csv file so that it is not necessary to keep pulling data from the Census website

In [1]:
#Data saved for 2017, 2018, 2019, 2020
for i in range(2017, 2021):
    extract_data(str(i))

NameError: name 'extract_data' is not defined

The data frame is then loaded from the cvs file for cleaning using the *load_data* function. Our initial data exploration showed that many of the variables provide the same information: one in the form of a code and the other in the form of a description. In addition, some of the variables were deemed irrelevant or non-meaningful. These variables were collected onto a *columns_to_drop* list and used in the *clean_data* function to remove those columns. A dictionary was also created to translate the meaning of the column names assigned by the Census and is used to rename the columns. Lastly, the cleaned data is also saved as a new csv file and ready to be used for data visualization

In [ ]:
for i in range(2017,2021):
    clean_data(load_data(str(i)),str(i))

In [ ]:
data_17 = load_data('2017_clean')
print(data_17.info())
data_18 = load_data('2018_clean')
print(data_18.info())
data_19 = load_data('2019_clean')
print(data_19.info())
data_20 = load_data('2020_clean')
print(data_20.info())

##### Characteristics of Module Business Characteristics

The Module of Business Charachteristics is a dataset accessible via the Annual Business Survey (ABS) API which contains estimates provided for employer firms that responded to a wide-ranging survey conducted by the US Census Bureau in 2021. In order to extract a sufficient amount of data, one must first import the following packages:

In [ ]:
import requests
import pandas as pd
import time

Next, we will isolate the columns that have deemed useful for the research conducted by our group by explicitly defining them in a list. Notice: the variable "get" is a near verbatim copy of the archtypical get statement provided by the ABS for this specific module with the only deviations being the exclusion of variables who's suffixies include include either an "S" or the characters "PCT." We have the following:

In [ ]:
useful_columns = ['NAME', 'NAICS2017_LABEL', "NAICS2017_F", 'SEX_LABEL', 'ETH_GROUP_LABEL', 'RACE_GROUP_LABEL', 'VET GROUP_LABEL', 'QDESC_LABEL', 'BUSCHAR_LABEL', 'BUSCHAR', 'YEAR', 'FIRMPDEMP', "FIRMPDEMP_F", 'RCPPDEMP', "RCPPDEMP_F", 'EMP', 'EMP_F', 'EMPSZFI', "URSZFI", "PAYANN", "PAYANN_F"]
get = "GEO_ID,NAME,NAICS2017,NAICS2017_F,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,URSZFI,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F"
get_list = get.split(',')

Now we shall designate the columns which we intend to exclude from our soon to be constructed Pandas DataFrame():

In [ ]:
# Appending the less useful columns
columns_to_drop = []
for items in get_list:
    if items in useful_columns:
        pass
    else:
        columns_to_drop.append(items)

With the sheep and goats seperated from our list of variables, we will now patch together our dataframe:

In [ ]:
# Module Business Characteristics
#========================================================================== 
#     What was tricky about this dataset was that the API key would not 
#     accept an open API call without yieleding a non-descript error which
#     it would send back to the administrator. However, the API worked fine 
#     when individual questions from the QDESC_LABEL were called. To work 
#     around this problem, we will transcribe the QDESC_LIST codes and 
#     loop through each call to the API, each time appending the new 
#     results to an established dataframe.
# ===========================================================================
qdesc_list =    [
                "AREMPT", "BUSTARDIFF", "BUSTARGETS", "CLOUDSERV", 
                "COVIDEFF", "COVIDFA", "COVIDOP", "CREDAPP", 
                "CREDREC", "CREDSOUR", "CUSSATMON", "DEBT", 
                "DIGBUSACT", "EMPTRE", "EMPTRP", "FINUSES", 
                "GOVFAREC", "GOVFAREQ","GOVFOR", "HEMPTR", 
                "KPIFREQ", "KPINUM", "PROMOTION", "SERVICEPROB", 
                "TECHUSE", "UNDERPERFORM",
                ]

for i in range (0, len(qdesc_list)):
    # Establishing the base case
    if i == 0:
        key = "ea9a7f5b5cf50ba715aea3b71320a0ca918b8233"
        geography = 'us'
        qdesc = qdesc_list[i]
        url = (f"https://api.census.gov/data/2020/absmcb?get={get}" +
                f"&for=us:" +
                f"*&QDESC_LABEL={qdesc}" +
                f"&key={key}")
        r = requests.get(url)
        text = r.text
        text
        
        # We are establishing a temporary dataframe with the text from the API call
        df = pd.read_json(text)

        # We're now extracting the headers from the 0th row
        headers = []
        for i in range(0,len(get_list)+ 1):
                headers.append(df.iloc[0][i])

        # Now we are extracting the rest of the data and updating the headers
        module_business_characteristics = pd.DataFrame(data = df.iloc[1:])
        for i in range(0,len(get_list)):
                module_business_characteristics.rename(columns = {i: headers[i]}, inplace = True)

        module_business_characteristics.drop(columns_to_drop, axis = 1, inplace = True)

        # Now we will cease operations for 10 seconds so we don't lose connection with the API
        time.sleep(10)
    else:
        # We will repeat the previous steps for the next QDESC_LABEL code
        key = "ea9a7f5b5cf50ba715aea3b71320a0ca918b8233"
        geography = 'us'
        qdesc = qdesc_list[i]
        url = (f"https://api.census.gov/data/2020/absmcb?get={get}" +
                f"&for=us:" +
                f"*&QDESC_LABEL={qdesc}" +
                f"&key={key}")

        r = requests.get(url)
        text = r.text

        df = pd.read_json(text)

        headers = []
        for i in range(0,len(get_list)+ 1):
                headers.append(df.iloc[0][i])

        module_business_characteristics_tmp = pd.DataFrame(data = df.iloc[1:])
        for i in range(0,len(get_list)):
                module_business_characteristics_tmp.rename(columns = {i: headers[i]}, inplace = True)

        module_business_characteristics_tmp.drop(columns_to_drop, axis = 1, inplace = True)

        # Now we will append this new data to the established dataframe
        module_business_characteristics = pd.DataFrame(module_business_characteristics.append(module_business_characteristics_tmp, ignore_index = True))
        # Of course, sleep for another 10 seconds to avoid flagging from the API
        time.sleep(10)

# Execution time: 25mins < Wait Time < 35mins


As evidenced by the use of the time.sleep() function, one may intuit that this is a large dataset. Indeed, the resulting dataframe contains greater than four million rows of data. Before proceeding further, when one requests information thourgh the ABS API, for every filteration parameter (in the code above we filter results by "QDESC_LABEL") one additional column is added. As well, regardless of the number of filtration parameters specified, I have found that the API will always return one additional column. We will now drop these two unnecessary columns.

In [ ]:
module_business_characteristics.drop([28, 29], axis = 1, inplace=True)

Now that we have successfully imported, and transformed the Module of Business Characteristics dataset, our final step is to save the dataframe in your preferred format. I chose to save this dataframe as a csv:

In [ ]:
module_business_characteristics.to_csv("module_business_characteristics.csv", mode = 'w')

##### Characteristics of a Business

The Census Annual Business Survey conveys information on economic and demographic characteristics for businesses in the United States. In order to work with this particular dataset, we have to import certain packages like requests and beautifulsoup which is necessary for web scrapping and pandas to work with the data:


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

After importing the necessary packages, we decided to only import columns that would be informational from the census data. Thus, instead of dropping them later on we only copied the columns in the get statement that did not include 'S' at the end or columns that were flagged.  

In [ ]:
useful_columns = ["NAME","NAICS2017_LABEL","BUSCHAR_LABEL","BUSCHAR","QDESC_LABEL","YIBSZFI_LABEL","EMP","EMPSZFI_LABEL","FIRMPDEMP","PAYANN","RCPPDEMP","RCPSZFI_LABEL","YEAR","VET_GROUP_LABEL","SEX_LABEL","RACE_GROUP_LABEL","ETH_GROUP_LABEL"]
get = "GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YIBSZFI_LABEL,EMPSZFI_LABEL,RCPSZFI_LABEL,YEAR,FIRMPDEMP,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F"
get_list = get.split(',')

In [ ]:
# Appending the less useful columns
columns_to_drop = []
for items in get_list:
    if items in useful_columns:
        pass
    else:
        columns_to_drop.append(items)
# When one adds filters to the api, it  adds duplicates of the columns to the data.
# Instead of renaming those columns (which are auto-filled in Pandas as integers)
# we will append the colmun number to drop it later 
columns_to_drop.append(49)

For this dataset, the years 2017,2018, 2019, and 2020 were loaded in. Next, the data was read as json then changed into a dataframe. After the remaining data was extracted and the first row was deemed column headers. Lastly, since the data is large time sleep was used so the API won't flag our data. It also took around 15 minutes to load the data.

In [ ]:
# Characteristics of a Business
for i in range(2017,2021):
        # Establishing the base case
        if i == 2017:
                key = "aefa14e56c15213f9543a02f0f5478021c972174"
                geography = 'us'
                url = (f"https://api.census.gov/data/2017/abscb?get={get}" +
                        f"&for={geography}:" +
                        f"*&key={key}")
                r = requests.get(url)
                text = r.text

                df = pd.read_json(text)

                # This selects the 0th-row of the data frame and loads it into an empty list
                headers = []
                for i in range(0,len(get_list)+ 1):
                        headers.append(df.iloc[0][i])

                # We're now extracting the remaining data and installing the correct column headers
                characteristics_of_a_business = pd.DataFrame(data = df.iloc[1:])
                for i in range(0,len(get_list)):
                        characteristics_of_a_business.rename(columns = {i: headers[i]}, inplace = True)

                characteristics_of_a_business.drop(columns_to_drop, axis = 1, inplace = True)
        # Proceeding with the other years
        else:
                year = str(i)
                key = "aefa14e56c15213f9543a02f0f5478021c972174"
                geography = 'us'
                url = (f"https://api.census.gov/data/{year}/abscb?get={get}" +
                        f"&for={geography}:" +
                        f"*&key={key}")
                r = requests.get(url)
                text = r.text

                df = pd.read_json(text)

                headers = []
                for i in range(0,len(get_list)+ 1):
                        headers.append(df.iloc[0][i])

                characteristics_of_a_business_tmp = pd.DataFrame(data = df.iloc[1:])
                for i in range(0,len(get_list)):
                        characteristics_of_a_business_tmp.rename(columns = {i: headers[i]}, inplace = True)

                characteristics_of_a_business_tmp.drop(columns_to_drop, axis = 1, inplace = True)
                
                # We're appending the next year's data onto the previous year's data frame
                characteristics_of_a_business = pd.DataFrame(characteristics_of_a_business.append(characteristics_of_a_business_tmp, ignore_index = True))
                
                # We'll pause for 5 seconds so the API won't flag us
                time.sleep(5)
                
# Execution Time: 10mins < Wait Time < 15mins


Finally, the data is loaded and saved as a new csv file.

In [ ]:
characteristics_of_a_business

In [ ]:
characteristics_of_a_business.to_csv("data/characteristics-of-a-business.csv", mode = 'w')